# Memory

In [1]:
from secret_key import hugging_facehub_key
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hugging_facehub_key

In [2]:
from langchain import HuggingFaceHub
# Define LLM
repo_id = "google/flan-t5-large"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64}
)

C:\Users\vijay\anaconda3\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


# Chat Message History

In [3]:
from langchain.memory import ChatMessageHistory

In [4]:
history = ChatMessageHistory()

history.add_user_message("hi!")

history.add_ai_message("whats up?")

In [5]:
history.messages

[HumanMessage(content='hi!', additional_kwargs={}, example=False),
 AIMessage(content='whats up?', additional_kwargs={}, example=False)]

In [6]:
history.add_user_message("Fine, what about you?")
history.messages

[HumanMessage(content='hi!', additional_kwargs={}, example=False),
 AIMessage(content='whats up?', additional_kwargs={}, example=False),
 HumanMessage(content='Fine, what about you?', additional_kwargs={}, example=False)]

In [7]:
from langchain import PromptTemplate, LLMChain

In [8]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=llm)


In [9]:
question = "Which city does the company's headquarters for our international employees reside in?"
print(llm_chain.run(question))

The company's headquarters for our international employees reside in New York City. New York City is a city. New York City is a city. So the answer is New York.


## ConversationBufferMemory

In [10]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [11]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


'Andrew: Hello, how can I help you?'

In [12]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Andrew: Hello, how can I help you?
Human: What is 1+1?
AI:

> Finished chain.


'Andrew: 1+1 is a number that is a combination of two numbers.'

In [13]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Andrew: Hello, how can I help you?
Human: What is 1+1?
AI: Andrew: 1+1 is a number that is a combination of two numbers.
Human: What is my name?
AI:

> Finished chain.


'Andrew: Andrew is a human name.'

In [14]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Andrew: Hello, how can I help you?
Human: What is 1+1?
AI: Andrew: 1+1 is a number that is a combination of two numbers.
Human: What is my name?
AI: Andrew: Andrew is a human name.


In [15]:
memory.load_memory_variables({})

{'history': 'Human: Hi, my name is Andrew\nAI: Andrew: Hello, how can I help you?\nHuman: What is 1+1?\nAI: Andrew: 1+1 is a number that is a combination of two numbers.\nHuman: What is my name?\nAI: Andrew: Andrew is a human name.'}

In [16]:
memory = ConversationBufferMemory()

In [17]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [18]:
print(memory.buffer)

Human: Hi
AI: What's up


In [19]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [20]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [21]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [22]:
from langchain.memory import ConversationBufferWindowMemory

In [23]:
memory = ConversationBufferWindowMemory(k=1)               

In [24]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [25]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [26]:
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [27]:
conversation.predict(input="Hi, my name is Andrew")

'Andrew: Hello, how can I help you?'

In [28]:
conversation.predict(input="What is 1+1?")

'Andrew: 1+1 is a number that is a combination of two numbers.'

In [29]:
conversation.predict(input="What is my name?")

'Andrew: Andrew is a fictional character.'

## ConversationTokenBufferMemory

In [30]:
#!pip install tiktoken

In [31]:
from langchain.memory import ConversationTokenBufferMemory

In [32]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

C:\Users\vijay\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [33]:
memory.load_memory_variables({})

{'history': 'Human: AI is what?!\nAI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [34]:
from langchain.memory import ConversationSummaryBufferMemory

In [35]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [36]:
memory.load_memory_variables({})

{'history': 'System: AI is hanging out.\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.'}

In [37]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [38]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: AI is hanging out.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'I would like to show you the latest demo of the latest LLM demo.'

In [39]:
memory.load_memory_variables({})

{'history': 'System: You have a day off tomorrow.\nHuman: What would be a good demo to show?\nAI: I would like to show you the latest demo of the latest LLM demo.'}